In [836]:
# !pip install praw
# !pip install bert-serving-server  # server
# !pip install bert-serving-client  # client, independent of `bert-serving-server`

     |████████████████████████████████| 61 kB 800 kB/s eta 0:00:011
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=fcb815c53855cba00de736ed8d5c21649106da0b4f76777fbd8c127b3e476a44
  Stored in directory: /Users/duytienpham/Library/Caches/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built GPUtil


In [840]:
# from bert_serving.client import BertClient
# from sklearn.cluster import KMeans

In [1]:
import praw
import pandas as pd
import datetime as dt
import numpy as np
import re
import multiprocessing
# from bert_serving.client i mport BertClient
from sklearn.cluster import KMeans

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors


In [2]:
reddit = praw.Reddit(client_id='lHajDv_1TYjNaQ',
                     client_secret='L_2GRacn5y2VDk2EYjoFK4IZv5I',
                     user_agent='covid_mental_health')

In [3]:
reddit

In [4]:
mental_health = reddit.subreddit('COVID19_support')

In [5]:
print(mental_health.description)

**Welcome to /r/COVID19_Support**
-
[Official Site](http://covidsupport.com)
---

[Familiarize yourself with our rules here. Thanks.](https://www.reddit.com/r/COVID19_support/about/rules/)
-

---

**Useful Resources:**

http://covidsupport.com/





---

**Related Subreddits:**
-

- /r/COVID19

- /r/Coronavirus

- /r/COVID19positive  

- /r/CoronavirusSupport

- /r/COVAnonymous

- /r/depression

- /r/SuicideWatch

- /r/SWResources

- /r/anxiety

- /r/Needafriend  

&nbsp;  

**Reference Links**  
-

- [Data is Beautiful - Easy to understand visualisations of the latest statistics](https://informationisbeautiful.net/visualizations/covid-19-coronavirus-infographic-datapack/)  

- [Our World in Data](https://ourworldindata.org/coronavirus#what-do-we-know-about-the-risk-of-dying-from-covid-19)  

- [Imperial College London -
Information, explanations and videos](https://www.imperial.ac.uk/mrc-global-infectious-disease-analysis/news--wuhan-coronavirus/)  

- [Information on how and when (an

In [6]:

posts = []
ml_subreddit = reddit.subreddit('COVID19_support')
for post in ml_subreddit.hot(limit=None):
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
covid19_support = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
covid19_support

,title,score,id,subreddit,url,num_comments,body,created
0,"NEW Welcome, Approvals and Flairs",62,fprwv0,COVID19_support,https://www.reddit.com/r/COVID19_support/comme...,278,**New Users**\n\nPlease contact the moderators...,1.585318e+09
1,WEEKLY GOOD NEWS STICKY: 5-11 July 2020,11,hlvg20,COVID19_support,https://www.reddit.com/r/COVID19_support/comme...,1,I hope everyone here had as good a weekend as ...,1.594016e+09
2,Seeing my friends with long haul post-COVID sy...,119,hptkhv,COVID19_support,https://www.reddit.com/r/COVID19_support/comme...,16,"Hey all,\n\nI am a contact tracer working on t...",1.594585e+09
3,My father just tested positive. He is 63 and h...,5,hpwzfm,COVID19_support,https://www.reddit.com/r/COVID19_support/comme...,2,"I know this is rediculous, I know that no one ...",1.594599e+09
4,School Anxiety,13,hprqte,COVID19_support,https://www.reddit.com/r/COVID19_support/comme...,2,Schools open back for the fall semester about ...,1.594575e+09
...,...,...,...,...,...,...,...,...
985,the little routines that i miss the most,16,gd62i3,COVID19_support,https://www.reddit.com/r/COVID19_support/comme...,9,the pandemic feels like it's never going to en...,1.588601e+09
986,I finally just laid in bed and cried all day,103,gcwdxp,COVID19_support,https://www.reddit.com/r/COVID19_support/comme...,19,That is all. And I’m still doing it.,1.588562e+09
987,A Letter From My Doctor #2: My physician has b...,215,gcrfkd,COVID19_support,https://drive.google.com/file/d/1veW7zkVdFCMlp...,34,,1.588545e+09
988,Can anyone give me tips on how to cope with th...,13,gd5lbs,COVID19_support,https://www.reddit.com/r/COVID19_support/comme...,6,"I’ve been exercising daily, & I work from home...",1.588599e+09


In [7]:
def format_date(created):
    return dt.datetime.fromtimestamp(created)
# covid19_support['created'] = covid19_support['created'].apply(get_date)

In [8]:
covid19_support.head()

,title,score,id,subreddit,url,num_comments,body,created
0,"NEW Welcome, Approvals and Flairs",62,fprwv0,COVID19_support,https://www.reddit.com/r/COVID19_support/comme...,278,**New Users**\n\nPlease contact the moderators...,1.585318e+09
1,WEEKLY GOOD NEWS STICKY: 5-11 July 2020,11,hlvg20,COVID19_support,https://www.reddit.com/r/COVID19_support/comme...,1,I hope everyone here had as good a weekend as ...,1.594016e+09
2,Seeing my friends with long haul post-COVID sy...,119,hptkhv,COVID19_support,https://www.reddit.com/r/COVID19_support/comme...,16,"Hey all,\n\nI am a contact tracer working on t...",1.594585e+09
3,My father just tested positive. He is 63 and h...,5,hpwzfm,COVID19_support,https://www.reddit.com/r/COVID19_support/comme...,2,"I know this is rediculous, I know that no one ...",1.594599e+09
4,School Anxiety,13,hprqte,COVID19_support,https://www.reddit.com/r/COVID19_support/comme...,2,Schools open back for the fall semester about ...,1.594575e+09


In [9]:
covid19_support['created'] = covid19_support['created'].apply(format_date) 

In [10]:
pd.DataFrame.to_csv(covid19_support, 'reddit_covid.csv')

# 2) Crawling parent comments for each post - Experiment with one link

In [13]:
from praw.models import MoreComments

submission = reddit.submission(url="https://www.reddit.com/r/COVID19_support/comments/fprwv0/new_welcome_approvals_and_flairs/")

In [14]:
submission.comments.replace_more(limit=None)
submission.comments[2].replies[0]


Comment(id='flyu14f')

In [172]:
len(submission.comments)

175

In [169]:
# submission.comments[0].body

In [250]:
type(format_date(submission.comments[0].created).strftime("%c"))

str

In [15]:
# example_dict = {}
# parent = submission.comments[2]
# id_ = parent.id
# example_dict[id_] = parent.body

#apply recursion to get child's child comment - similar to factorial
def get_replies(parent, dict_):
    children = parent.replies
    if len(children) > 0:
        for child in children:
            dict_[child.id] = child.body + ' ***** {}'.format(format_date(child.created)) #.strftime('%c'))
            get_replies(child, dict_)
            
    return dict_



In [785]:
id_comment_dict = {}
# id_date_dict = {}
urls = covid19_support.url.tolist()[:1] # get a list of url to iterate
for url in urls:
    print(url)
    #initialize the thread of subreddit - set replace more to collect all comments as we may encounter an error due to 'More Comments'
    thread = reddit.submission(url = url)
    thread.comments.replace_more(limit = None)
    
    
    #get the number of all 1-level-down child comments
    length = len(thread.comments)
    
    for i in range(length):
        parent = thread.comments[i]
        id_ = parent.id
        id_comment_dict[id_] = parent.body + ' ***** {}'.format(format_date(parent.created).strftime('%c'))
        get_replies(parent, id_comment_dict)
#         get_date(parent, id_date_dict)

    

https://www.reddit.com/r/COVID19_support/comments/fprwv0/new_welcome_approvals_and_flairs/


In [793]:
# pd.DataFrame(id_comment_dict.items(), columns = ['id', 'comment'])
# id_comment_dict

,id,comment
0,flwyo6p,Dear admins: blocking comments on discussions ...
1,flz4qqf,Yes! Masks are simply barriers to cover people...
2,fmbwb0n,"Also, the fact that the virus stays active on ..."
3,fmq314b,I need to jump in here and clarify something. ...
4,fmwwzt1,"Yes! Also,Please consider there are East Asian..."
...,...,...
272,fxcgxkp,Can I have approval to post? ***** Wed Jul 8 ...
273,fm6k7cy,Can I get permission to post? ***** Thu Apr 2...
274,fnnobib,[removed] ***** Fri Apr 17 08:20:14 2020
275,fno54d4,"We have nothing to hide on this sub, and are m..."


In [18]:
#get all comments and replies from all post in COVID19_support

id_comment_dict = {}
# id_date_dict = {}
urls = covid19_support.url.tolist() # get a list of url to iterate
for url in urls:
    print(url)
    #initialize the thread of subreddit - set replace more to collect all comments as we may encounter an error due to 'More Comments'
    try:
        thread = reddit.submission(url = url)
        thread.comments.replace_more(limit = None)


        #get the number of all 1-level-down child comments
        length = len(thread.comments)

        for i in range(length):
            parent = thread.comments[i]
            id_ = parent.id
            id_comment_dict[id_] = parent.body + ' ***** {}'.format(format_date(parent.created).strftime('%c'))
            get_replies(parent, id_comment_dict)
    #         get_date(parent, id_date_dict)
    
    except:
        print('Not a reddit link: {}'.format(str(url)))

https://www.reddit.com/r/COVID19_support/comments/fprwv0/new_welcome_approvals_and_flairs/
https://www.reddit.com/r/COVID19_support/comments/hlvg20/weekly_good_news_sticky_511_july_2020/
https://www.reddit.com/r/COVID19_support/comments/hptkhv/seeing_my_friends_with_long_haul_postcovid/
https://www.reddit.com/r/COVID19_support/comments/hpwzfm/my_father_just_tested_positive_he_is_63_and_has/
https://www.reddit.com/r/COVID19_support/comments/hprqte/school_anxiety/
https://www.reddit.com/r/COVID19_support/comments/hpesty/my_58yo_esthetician_mother_with_kidney_disease_no/
https://www.reddit.com/r/COVID19_support/comments/hpuiam/feel_like_i_cant_do_this/
https://www.reddit.com/r/COVID19_support/comments/hpvcbz/having_workplace_issues_who_exactly_do_i_contact/
https://www.reddit.com/r/COVID19_support/comments/hpp9lz/covid19_and_loneliness/
https://www.reddit.com/r/COVID19_support/comments/hpxxhv/is_there_any_hope_for_the_us/
https://www.reddit.com/r/COVID19_support/comments/hptf1x/is_germx_a

In [20]:
comment_df = pd.DataFrame(id_comment_dict.items(), columns = ['id', 'comment'])
comment_df

,id,comment
0,flwyo6p,Dear admins: blocking comments on discussions ...
1,flz4qqf,Yes! Masks are simply barriers to cover people...
2,fmbwb0n,"Also, the fact that the virus stays active on ..."
3,fmq314b,I need to jump in here and clarify something. ...
4,fmwwzt1,"Yes! Also,Please consider there are East Asian..."
...,...,...
13390,fpheoeb,Meditation is a great idea. If meditation itse...
13391,fphw0xw,Thank you; I find meditation makes me worse. T...
13392,fpg97gt,I don't know if there is an official document ...
13393,fpgarzx,"This is fantastic, thanks. I'll definitely sha..."


In [21]:
# pd.DataFrame.to_csv(comment_df, 'reddit_COVID19_support_thread_comments.csv')

#### checking method to see if every comment and reply has a date followed with ' ***** '

In [259]:
# for key in list(id_comment_dict.keys()):
#     print('ID: {} - Comment: {}'.format(key,id_comment_dict[key]))
#     print('_____________________________________________________________________ \n\n')

# 3) Create dataframe which has id and parent_id for each comment and reply

#### Problem: each comment has a specific id - not user! example: submission.comments[2] - Kreslin

In [62]:
# #apply recursion to get child's child comment - similar to factorial
# def recurse_id(parent):
#     ''' Function: use recursion to get child comments for each comment
#     Example:
#         {'flz4qqf': ['fmbwb0n', 'fmwwzt1', 'flzg2dw', 'fw8yicz', 'fmq314b'],
#      'fmbwb0n': ['fmq314b'],
#      'fmq314b': [],
#      'fmwwzt1': [],
#      'flzg2dw': [],
#      'fw8yicz': [],
#      'fnjo73v': [],
#      'fohxngv': [],
#      'flyu5li': ['flzjzbi', 'fmq0cb3'],
#      'flzjzbi': [],
#      'fmq0cb3': []}
#     '''
#     children_id = parent.replies
#     if len(children_id) > 0:
#         for child_id in children_id:
#             id_dict[child_id.id] = [id_.id for id_ in child_id.replies.list()]
#             recurse_id(child_id)
            
# #     print('recursive: {}'.format(recurse_id))
#     return id_dict

# def get_parent_child_id(parent):
#     ''' Function: since recurse_id function get ALL replies (even the indirect chilren of its children) 
#     Goal: get dictionary with KEY:the parent and VALUES: DIRECT child node
#     Example:
#         {'flz4qqf': ['fmbwb0n', 'fmwwzt1', 'flzg2dw', 'fw8yicz'],
#      'fmbwb0n': ['fmq314b'],
#      'fmq314b': [],
#      'fmwwzt1': [],
#      'flzg2dw': [],
#      'fw8yicz': [],
#      'fnjo73v': [],
#      'fohxngv': [],
#      'flyu5li': ['flzjzbi', 'fmq0cb3'],
#      'flzjzbi': [],
#      'fmq0cb3': []}
#     '''
#     dict_ = recurse_id(parent)
#     keys = list(dict_.keys())
#     for key in keys:
#         vals = list(dict_[key])
#         if len(vals) > 1:
#             lst = list(dict_[key]) 
#             for val1 in lst:
#                 for val2 in lst:
#                     print(val)
#                     if (val1 != val2) and (val2 in dict_[val1]):
#                         dict_[key].remove(val2)
                        
#     return(dict_)

# def get_inverted_dict(dict_):
#     ''' Function: invert dictionary value in the original dictionary will become key
    
#     Example:
#     Original: 
#         {'flz4qqf': ['fmbwb0n', 'fmwwzt1', 'flzg2dw', 'fw8yicz', 'fmq314b'],
#      'fmbwb0n': ['fmq314b'],
#      'fmq314b': [],
#      'fmwwzt1': [],
#      'flzg2dw': [],
#      'fw8yicz': [],
#      'fnjo73v': [],
#      'fohxngv': [],
#      'flyu5li': ['flzjzbi', 'fmq0cb3'],
#      'flzjzbi': [],
#      'fmq0cb3': []}
     
#      Return:
#          {'fmbwb0n': ['flz4qqf'],
#      'fmwwzt1': ['flz4qqf'],
#      'flzg2dw': ['flz4qqf'],
#      'fw8yicz': ['flz4qqf'],
#      'fmq314b': ['fmbwb0n'],
#      'flzjzbi': ['flyu5li'],
#      'fmq0cb3': ['flyu5li']}
     
#     '''
#     new_dic = {}
#     for k,v in dict_.items():
#         for x in v:
#             new_dic.setdefault(x,[]).append(k)
            
#     return new_dic

In [87]:
def recurse_id(parent, id_dict):
    ''' Function: use recursion to get child comments for each comment
    Example:
        {'flz4qqf': ['fmbwb0n', 'fmwwzt1', 'flzg2dw', 'fw8yicz', 'fmq314b'],
     'fmbwb0n': ['fmq314b'],
     'fmq314b': [],
     'fmwwzt1': [],
     'flzg2dw': [],
     'fw8yicz': [],
     'fnjo73v': [],
     'fohxngv': [],
     'flyu5li': ['flzjzbi', 'fmq0cb3'],
     'flzjzbi': [],
     'fmq0cb3': []}
    '''
#     id_dict = {}
    children_id = parent.replies
    if len(children_id) > 0:
        for child_id in children_id:
            id_dict[child_id.id] = [id_.id for id_ in child_id.replies.list()]
            recurse_id(child_id,id_dict)
            
#     print('recursive: {}'.format(recurse_id))
    return id_dict

# **********************************************************************************************************************************************************************************

def get_parent_child_id(parents, id_dict):
    ''' Function: since recurse_id function get ALL replies (even the indirect chilren of its children) 
    Goal: get dictionary with KEY:the parent and VALUES: DIRECT child node
    Example:
        {'flz4qqf': ['fmbwb0n', 'fmwwzt1', 'flzg2dw', 'fw8yicz'],
     'fmbwb0n': ['fmq314b'],
     'fmq314b': [],
     'fmwwzt1': [],
     'flzg2dw': [],
     'fw8yicz': [],
     'fnjo73v': [],
     'fohxngv': [],
     'flyu5li': ['flzjzbi', 'fmq0cb3'],
     'flzjzbi': [],
     'fmq0cb3': []}
    '''
    answer = {}
    for parent in parents:
        example = recurse_id(parent, id_dict)
        keys = list(example.keys())
        for key in keys:
            vals = list(example[key])
            if len(vals) > 1:
                lst = list(example[key]) 
                for val1 in lst:
                    for val2 in lst:
                        if (val1 != val2) and (val2 in example[val1]) and (val2 in example[key]):
                            example[key].remove(val2)
        if bool(example) == True:
            answer.update(example)
        example.clear()
    return answer

# **********************************************************************************************************************************************************************************

def get_inverted_dict(dict_):
    ''' Function: invert dictionary value in the original dictionary will become key
    
    Example:
    Original: 
        {'flz4qqf': ['fmbwb0n', 'fmwwzt1', 'flzg2dw', 'fw8yicz', 'fmq314b'],
     'fmbwb0n': ['fmq314b'],
     'fmq314b': [],
     'fmwwzt1': [],
     'flzg2dw': [],
     'fw8yicz': [],
     'fnjo73v': [],
     'fohxngv': [],
     'flyu5li': ['flzjzbi', 'fmq0cb3'],
     'flzjzbi': [],
     'fmq0cb3': []}
     
     Return:
         {'fmbwb0n': ['flz4qqf'],
     'fmwwzt1': ['flz4qqf'],
     'flzg2dw': ['flz4qqf'],
     'fw8yicz': ['flz4qqf'],
     'fmq314b': ['fmbwb0n'],
     'flzjzbi': ['flyu5li'],
     'fmq0cb3': ['flyu5li']}
     
    '''
    new_dic = {}
    for k,v in dict_.items():
        for x in v:
            new_dic.setdefault(x,[]).append(k)
            
    return new_dic

# **********************************************************************************************************************************************************************************



#### Code below - used for testing recurse_id and get_child_parent_id

In [85]:
# submission = reddit.submission(url="https://www.reddit.com/r/COVID19_support/comments/fprwv0/new_welcome_approvals_and_flairs/")
# parents = [id_ for id_ in submission.comments.list()] #get a list of parents
# # parent = parents[4]
# # example.clear()
# answer = {}
# for parent in parents[:1]:
#     empty_dict = {}
#     example = recurse_id(parent, empty_dict)
#     print(example)
#     keys = list(example.keys())
#     for key in keys:
#         vals = list(example[key])
#         if len(vals) > 1:
#             lst = list(example[key]) 
#             for val1 in lst:
#                 for val2 in lst:
#                     if (val1 != val2) and (val2 in example[val1]) and (val2 in example[key]):
#                         example[key].remove(val2)
#     if bool(example) == True:
#         answer.update(example)
#     example.clear()
# #         print(example)
# #         print('\n\n')

# Put things together - iterate through several head comments of each thread in COVID19_support sunreddit

In [121]:
urls = covid19_support.url.tolist() 
id_parentID = {}
id_parent_df = pd.DataFrame()
for url in urls:
    if bool(re.match(r'^(https:\/\/www\.reddit\.com)', url)) == True:
        print(url)
        submission = reddit.submission(url = url)
        submission.comments.replace_more(limit = None)
        parents = [id_ for id_ in submission.comments.list()]

        #get the child-parent id for comments directly to original post
        for x in submission.comments.list():
            id_parentID.setdefault(x.id,[]).append(submission.id) #submission.id = the original post

        #get the child-parent id for replies to the comments
        id_dict = {}
        child_parent_matching = get_parent_child_id(parents, id_dict) #returns dictionary (parent-children)
        all_ids = list(child_parent_matching.keys()) # get the ids for ALL replies and comments - prepare for dataframe setup
        child_parent_matching = get_inverted_dict(child_parent_matching) #returns dictionary (child-parent)
        id_parentID.update(child_parent_matching)

        id_dict.clear()

        #prepare dataframe
        temp_df = pd.DataFrame(all_ids, columns = ['id'])

        temp_df['parent_id'] = temp_df['id'].apply(lambda x: child_parent_matching[x][0] if (x in child_parent_matching.keys()) else float('nan'))

        id_parent_df = pd.concat([id_parent_df,temp_df])

https://www.reddit.com/r/COVID19_support/comments/fprwv0/new_welcome_approvals_and_flairs/
https://www.reddit.com/r/COVID19_support/comments/hlvg20/weekly_good_news_sticky_511_july_2020/
https://www.reddit.com/r/COVID19_support/comments/hptkhv/seeing_my_friends_with_long_haul_postcovid/
https://www.reddit.com/r/COVID19_support/comments/hpwzfm/my_father_just_tested_positive_he_is_63_and_has/
https://www.reddit.com/r/COVID19_support/comments/hprqte/school_anxiety/
https://www.reddit.com/r/COVID19_support/comments/hpesty/my_58yo_esthetician_mother_with_kidney_disease_no/
https://www.reddit.com/r/COVID19_support/comments/hpuiam/feel_like_i_cant_do_this/
https://www.reddit.com/r/COVID19_support/comments/hpvcbz/having_workplace_issues_who_exactly_do_i_contact/
https://www.reddit.com/r/COVID19_support/comments/hpp9lz/covid19_and_loneliness/
https://www.reddit.com/r/COVID19_support/comments/hpxxhv/is_there_any_hope_for_the_us/
https://www.reddit.com/r/COVID19_support/comments/hptf1x/is_germx_a

In [122]:
id_parent_df #a table of id_parentID for the COVID19_support

,id,parent_id
0,flz4qqf,NaN
1,fmbwb0n,flz4qqf
2,fmq314b,fmbwb0n
3,fmwwzt1,flz4qqf
4,flzg2dw,flz4qqf
...,...,...
3,fpmxy69,fpmxprm
0,fpheoeb,NaN
1,fphw0xw,fpheoeb
0,fpgarzx,NaN


# Data Exploration

In [825]:
df = covid19_support.copy()

In [826]:
df

,title,score,id,subreddit,url,num_comments,body,created
0,"NEW Welcome, Approvals and Flairs",63,fprwv0,COVID19_support,https://www.reddit.com/r/COVID19_support/comme...,276,**New Users**\n\nPlease contact the moderators...,2020-03-27 10:13:04
1,WEEKLY GOOD NEWS STICKY: 5-11 July 2020,10,hlvg20,COVID19_support,https://www.reddit.com/r/COVID19_support/comme...,1,I hope everyone here had as good a weekend as ...,2020-07-06 02:17:41
2,Sudden intense anxiety due to impending change...,71,hoozlw,COVID19_support,https://www.reddit.com/r/COVID19_support/comme...,17,I'll try to keep this as organized as possible...,2020-07-10 17:31:42
3,Feeling more helpless with each day,6,houeqz,COVID19_support,https://www.reddit.com/r/COVID19_support/comme...,1,I’ve been depressed in the past but it hits an...,2020-07-10 22:35:04
4,Just read a really depressing article about ho...,3,hospp1,COVID19_support,https://www.reddit.com/r/COVID19_support/comme...,20,It basically said because Trump refuses to hav...,2020-07-10 21:04:59
5,Intense depression. Anyone else?,167,hob05u,COVID19_support,https://www.reddit.com/r/COVID19_support/comme...,66,I am just exhausted. I've gotten used to follo...,2020-07-10 00:47:34
6,COVID-19 has begun to rob me of my hope for th...,1,hp0h6i,COVID19_support,https://www.reddit.com/r/COVID19_support/comme...,0,I spent the last two weeks meticulously resear...,2020-07-11 04:23:18
7,Feeling dumb and scared...,1,hozqqi,COVID19_support,https://www.reddit.com/r/COVID19_support/comme...,0,"Hi,\nI went yesterday to get tested via nasal ...",2020-07-11 03:35:53
8,Older people are acting unaffected by this. Am...,2,houfpp,COVID19_support,https://www.reddit.com/r/COVID19_support/comme...,1,"My parents, grandparents, and friends’ parents...",2020-07-10 22:36:21
9,Been having shortness of breathe and lethargy,3,hoqi6h,COVID19_support,https://www.reddit.com/r/COVID19_support/comme...,6,"For the past 3 days, i've been having tightnes...",2020-07-10 19:03:47


# TItle-based analysis

In [828]:
def text_to_word_list(text):
    ''' Pre process and convert texts to a list of words 
    method inspired by method from eliorc github repo: https://github.com/eliorc/Medium/blob/master/MaLSTM.ipynb'''
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!?.\/'+]", " ", text)
    text = re.sub(r"\+", " plus ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\?", " ? ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text

In [833]:
title_df = df[['title']]
title_df['processed_title'] = title_df['title'].apply(text_to_word_list)

/Users/duytienpham/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### Using Word2Vec

In [883]:
sent = [row for row in title_df.processed_title]

In [884]:
phrases = Phrases(sent, min_count=1, progress_per=50000)
bigram = Phraser(phrases)
sentences = bigram[sent]
sentences[9]

['been', 'having', 'shortness', 'of', 'breathe', 'and', 'lethargy']

In [885]:
w2v_model = Word2Vec(min_count=3,
                     window=4,
                     size=300,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=multiprocessing.cpu_count()-1)


w2v_model.build_vocab(sentences, progress_per=50000)

In [886]:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(3, 3270)

In [887]:
#intialize word_vector
word_vector = w2v_model.wv

In [889]:
kmeans_model = KMeans(n_clusters=2, max_iter=1000, random_state=True, n_init=50).fit(X=word_vector.vectors.astype(np.float))

In [890]:
word_vector.similar_by_vector(kmeans_model.cluster_centers_[0], topn=10, restrict_vocab=None)

[('the', 0.748874306678772),
 ('and', 0.7402434349060059),
 ('?', 0.05519992113113403),
 ('t', 0.027403593063354492),
 ('i_can', -0.03561785817146301)]

In [891]:
positive_cluster_center = kmeans_model.cluster_centers_[0]
negative_cluster_center = kmeans_model.cluster_centers_[1]

In [892]:
words = pd.DataFrame(word_vector.vocab.keys())
words.columns = ['words']
words['vectors'] = words.words.apply(lambda x: word_vector.wv[f'{x}'])
words['vectors'] = words.vectors.apply(lambda x: x.astype(np.double))
# words['cluster'] = words.vectors.apply(lambda x: kmeans_model.predict([np.array(x)]))
# words.cluster = words.cluster.apply(lambda x: x[0])

/Users/duytienpham/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [893]:
kmeans_model.predict(words.vectors[0].reshape(1,-1))

array([0], dtype=int32)

In [894]:
words['cluster'] = words.vectors.apply(lambda x: kmeans_model.predict([np.array(x)]))
words.cluster = words.cluster.apply(lambda x: x[0])

In [896]:
words['cluster_value'] = [1 if i==0 else -1 for i in words.cluster]
words['closeness_score'] = words.apply(lambda x: 1/(kmeans_model.transform([x.vectors]).min()), axis=1)
words['sentiment_coeff'] = words.closeness_score * words.cluster_value

(5, 6)